# 🧱 Multi-Layer Land Suitability Screening for Construction
This notebook evaluates whether a given land parcel is suitable for construction, using satellite data and open geospatial datasets. It includes layers for soil, elevation, flood risk, zoning, infrastructure, and environmental protection.

✅ Powered by Google Earth Engine + Python.


In [ ]:
!pip install earthengine-api geemap folium

In [ ]:
import ee
import datetime
import folium
from ipywidgets import widgets
from IPython.display import display

ee.Authenticate()
ee.Initialize()

## 🗺️ AOI and Date Inputs

In [ ]:
min_lon = widgets.FloatText(value=103.6, description='Min Lon:')
min_lat = widgets.FloatText(value=1.2, description='Min Lat:')
max_lon = widgets.FloatText(value=103.8, description='Max Lon:')
max_lat = widgets.FloatText(value=1.4, description='Max Lat:')
start_date = widgets.DatePicker(description='Start', value=datetime.date(2023, 11, 1))
end_date = widgets.DatePicker(description='End', value=datetime.date(2023, 12, 1))
display(min_lon, min_lat, max_lon, max_lat, start_date, end_date)

## 🛰️ Multi-Layer Suitability Analysis

In [ ]:
def evaluate_suitability():
    aoi = ee.Geometry.Rectangle([
        min_lon.value, min_lat.value, max_lon.value, max_lat.value
    ])
    
    # Sentinel-1 for soil moisture
    s1 = ee.ImageCollection('COPERNICUS/S1_GRD') \
        .filterBounds(aoi) \
        .filterDate(str(start_date.value), str(end_date.value)) \
        .select(['VV', 'VH']) \
        .mean()

    # Sentinel-2 for NDVI
    s2 = ee.ImageCollection('COPERNICUS/S2_SR') \
        .filterBounds(aoi) \
        .filterDate(str(start_date.value), str(end_date.value)) \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)) \
        .select(['B4', 'B8', 'B11']) \
        .median()
    
    ndvi = s2.normalizedDifference(['B8', 'B4']).rename('NDVI')
    bare_soil = ndvi.lt(0.2).And(s2.select('B11').gt(1000))
    dry_soil = s1.select('VV').lt(-15).And(s1.select('VH').lt(-20))

    # DEM for slope
    dem = ee.Image('USGS/SRTMGL1_003')
    slope = ee.Terrain.slope(dem)
    flat = slope.lt(5)

    # Flood zones
    flood = ee.Image('JRC/GSW1_3/MaxExtent')
    no_flood = flood.clip(aoi).Not()

    # Road proximity (proxy infra)
    roads = ee.FeatureCollection('users/giswqs/public/osm_roads')
    near_road = roads.distance(1000).lt(1000)

    # Protected areas
    protected = ee.FeatureCollection('WCMC/WDPA/current/polygons')
    no_protected = protected.geometry().contains(aoi).Not()

    # Final Score
    score = bare_soil.And(dry_soil).multiply(0.3) \
        .add(flat.multiply(0.2)) \
        .add(no_flood.multiply(0.2)) \
        .add(near_road.multiply(0.15)) \
        .add(ee.Image.constant(no_protected).multiply(0.15))

    return score.visualize(min=0, max=1, palette=['red', 'yellow', 'green']), aoi

## 🗺️ Visualize Map

In [ ]:
def show_map():
    image, aoi = evaluate_suitability()
    center = aoi.centroid().coordinates().getInfo()[::-1]
    m = folium.Map(location=center, zoom_start=12)
    folium.TileLayer(
        tiles=image.getMapId()['tile_fetcher'].url_format,
        attr='Earth Engine', overlay=True, name='Suitability'
    ).add_to(m)
    folium.LayerControl().add_to(m)
    return m

show_map()